[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/show4dstem/show4dstem_all_features.ipynb)

# Show4DSTEM — All Features

Comprehensive demo of every Show4DSTEM feature using realistic synthetic 4D-STEM data
with a bright-field disk, Bragg reflections, Kikuchi-like background, and scan-position-dependent
diffraction contrast.

In [ ]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

## 1. Synthetic 4D-STEM Data Generator

Creates a realistic 4D-STEM dataset with:
- Bright-field disk with position-dependent beam tilt
- 6 first-order Bragg reflections with orientation-dependent intensity
- 6 second-order spots (weaker)
- Radial amorphous scattering background
- Poisson shot noise

In [ ]:
import numpy as np
from quantem.widget import Show4DSTEM


def make_4dstem(scan_x=16, scan_y=16, det_x=64, det_y=64):
    """4D-STEM dataset with BF disk, Bragg spots, and Kikuchi-like background."""
    data = np.zeros((scan_x, scan_y, det_x, det_y), dtype=np.float32)
    cy, cx = det_x // 2, det_y // 2
    yy, xx = np.mgrid[:det_x, :det_y]
    dist = np.sqrt((xx - cx)**2 + (yy - cy)**2)

    # Amorphous background (radial falloff)
    bg = 0.05 * np.exp(-dist / 30)

    for i in range(scan_x):
        for j in range(scan_y):
            dp = bg.copy()
            # BF disk with slight shift depending on scan position (beam tilt)
            shift_x = 0.3 * np.sin(2*np.pi*i/scan_x)
            shift_y = 0.3 * np.cos(2*np.pi*j/scan_y)
            bf_dist = np.sqrt((xx - cx - shift_x)**2 + (yy - cy - shift_y)**2)
            dp += np.where(bf_dist < 8, 1.0 + 0.2*np.cos(bf_dist*0.5), 0.0)

            # 6 first-order Bragg spots
            for k in range(6):
                angle = k * np.pi / 3
                sx = cx + 20 * np.cos(angle) + shift_x * 2
                sy = cy + 20 * np.sin(angle) + shift_y * 2
                # Intensity varies with scan position (thickness/orientation)
                intensity = 0.4 * (1 + 0.5*np.sin(2*np.pi*(i*np.cos(angle) + j*np.sin(angle))/scan_x))
                dp += intensity * np.exp(-((xx-sx)**2 + (yy-sy)**2) / (2*2.5**2))

            # Second-order spots (weaker)
            for k in range(6):
                angle = k * np.pi / 3 + np.pi / 6
                sx = cx + 35 * np.cos(angle)
                sy = cy + 35 * np.sin(angle)
                dp += 0.1 * np.exp(-((xx-sx)**2 + (yy-sy)**2) / (2*2**2))

            # Shot noise
            dp = np.maximum(dp, 0)
            dp = np.random.poisson(np.clip(dp * 200, 0, 1e6)).astype(np.float32) / 200
            data[i, j] = dp
    return data


data = make_4dstem(scan_x=16, scan_y=16, det_x=64, det_y=64)
print(f"Shape: {data.shape}, dtype: {data.dtype}")
print(f"Range: [{data.min():.3f}, {data.max():.3f}]")

## 2. Basic 4D-STEM Viewer

Default view with auto-detected BF disk center and radius.

In [ ]:
w_basic = Show4DSTEM(data)
w_basic

## 3. Flattened 3D Input with scan_shape

When data arrives as a flat stack `(N, det_x, det_y)` from a detector readout,
pass `scan_shape` to tell the widget how to reshape it into a 2D scan grid.

In [ ]:
# Flatten 4D -> 3D: (16*16, 64, 64) = (256, 64, 64)
data_flat = data.reshape(-1, data.shape[2], data.shape[3])
print(f"Flattened shape: {data_flat.shape}")

w_flat = Show4DSTEM(data_flat, scan_shape=(16, 16))
w_flat

## 4. Auto-Detect Center

The `auto_detect_center()` method uses centroid analysis of the summed diffraction
pattern to find the BF disk center and estimate its radius.

In [ ]:
w_auto = Show4DSTEM(data)
w_auto.auto_detect_center()
print(f"Auto-detected center: ({w_auto.center_row:.2f}, {w_auto.center_col:.2f})")
print(f"Auto-detected BF radius: {w_auto.bf_radius:.2f} px")
print(f"Detector center (expected): ({data.shape[2]//2}, {data.shape[3]//2})")
w_auto

## 5. Manual Center and BF Radius

Override auto-detection with explicit center and bright-field radius values.
Useful when the auto-detect does not work well (e.g., very noisy data or
off-axis diffraction patterns).

In [ ]:
w_manual = Show4DSTEM(data, center=(32.0, 32.0), bf_radius=9.0)
print(f"Manual center: ({w_manual.center_row}, {w_manual.center_col})")
print(f"Manual BF radius: {w_manual.bf_radius}")
w_manual

## 6. ROI Modes

Different ROI shapes for virtual imaging. Each mode integrates diffraction
intensity within the ROI to produce a virtual image in real-space.

- **point** — single pixel (fastest)
- **circle** — virtual BF detector
- **square** — square integration region
- **annular** — ADF/HAADF ring detector
- **rect** — rectangular region

In [ ]:
# Point ROI (single-pixel virtual image)
w_point = Show4DSTEM(data)
w_point.roi_point()
w_point

In [ ]:
# Circle ROI (virtual bright-field)
w_circle = Show4DSTEM(data)
w_circle.roi_circle(radius=8.0)
w_circle

In [ ]:
# Square ROI
w_square = Show4DSTEM(data)
w_square.roi_square(half_size=6.0)
w_square

In [ ]:
# Annular ROI (ADF — collects scattered electrons outside BF disk)
w_annular = Show4DSTEM(data)
w_annular.roi_annular(inner_radius=10.0, outer_radius=25.0)
w_annular

In [ ]:
# Rectangle ROI
w_rect = Show4DSTEM(data)
w_rect.roi_rect(width=20.0, height=10.0)
w_rect

## 7. Log Scale

Diffraction patterns have very high dynamic range. The bright BF disk can be
orders of magnitude brighter than Bragg spots. Log scale compresses this range
to reveal weak features.

In [ ]:
w_log = Show4DSTEM(data, log_scale=True)
w_log.roi_circle()
w_log

## 8. Precomputed Virtual Images

When `precompute_virtual_images=True`, the widget pre-calculates BF, ABF,
and ADF virtual images at startup. Switching between these presets is then
instantaneous (no recomputation needed).

In [ ]:
w_precomp = Show4DSTEM(data, precompute_virtual_images=True)
w_precomp.roi_circle()
print(f"BF cached: {w_precomp._cached_bf_virtual is not None}")
print(f"ABF cached: {w_precomp._cached_abf_virtual is not None}")
print(f"ADF cached: {w_precomp._cached_adf_virtual is not None}")
w_precomp

## 9. Raster Scan Animation

Animate the scan position across the sample in a raster pattern (row by row,
left to right), mimicking real STEM acquisition. The crosshair moves through
the virtual image while the diffraction pattern updates live.

In [ ]:
w_raster = Show4DSTEM(data)
w_raster.roi_circle(radius=8.0)
w_raster.raster(step=2, interval_ms=150, loop=True)
w_raster

## 10. Custom Path Animation

Define an arbitrary sequence of scan positions. Here we create a diagonal path
and a spiral path to explore specific regions of the sample.

In [ ]:
# Diagonal path from corner to corner
diagonal_path = [(i, i) for i in range(16)]

w_diag = Show4DSTEM(data)
w_diag.roi_circle(radius=8.0)
w_diag.set_path(diagonal_path, interval_ms=200, loop=True)
print(f"Diagonal path: {len(diagonal_path)} positions")
w_diag

In [ ]:
# Spiral path from center outward
spiral_path = []
cx, cy = 8, 8  # center of scan
for r in range(1, 8):
    n_pts = max(4, r * 4)
    for t in range(n_pts):
        angle = 2 * np.pi * t / n_pts
        x = int(round(cx + r * np.cos(angle)))
        y = int(round(cy + r * np.sin(angle)))
        if 0 <= x < 16 and 0 <= y < 16:
            spiral_path.append((x, y))

w_spiral = Show4DSTEM(data)
w_spiral.roi_circle(radius=8.0)
w_spiral.set_path(spiral_path, interval_ms=100, loop=True)
print(f"Spiral path: {len(spiral_path)} positions")
w_spiral

## 11. Playback Controls

Programmatic control of path animations: pause, play, stop, and jump to
specific positions.

In [ ]:
w_ctrl = Show4DSTEM(data)
w_ctrl.roi_circle(radius=8.0)
w_ctrl.raster(step=1, interval_ms=100)
w_ctrl

In [ ]:
# Pause the animation
w_ctrl.pause()
print(f"Paused at index: {w_ctrl.path_index}, position: {w_ctrl.position}")

In [ ]:
# Jump to a specific frame and resume
w_ctrl.goto(50)
print(f"Jumped to index: {w_ctrl.path_index}, position: {w_ctrl.position}")
w_ctrl.play()

In [ ]:
# Stop and reset to beginning
w_ctrl.stop()
print(f"Stopped at index: {w_ctrl.path_index}")

## 12. Scale Bars

Set `pixel_size` for real-space scale bar (in angstroms) and `k_pixel_size` for
k-space / diffraction scale bar (in milliradians).

In [ ]:
w_scale = Show4DSTEM(
    data,
    pixel_size=2.39,       # 2.39 angstrom per scan pixel
    k_pixel_size=0.46,     # 0.46 mrad per detector pixel
)
w_scale.roi_circle(radius=8.0)
print(f"Real-space pixel size: {w_scale.pixel_size} angstrom")
print(f"K-space pixel size: {w_scale.k_pixel_size} mrad")
print(f"K-space calibrated: {w_scale.k_calibrated}")
w_scale

## 13. Rectangular Scan Shape

Non-square scan grids are common when the scan region is not square.
Here we generate a 24x12 scan with the same diffraction physics.

In [ ]:
data_rect = make_4dstem(scan_x=24, scan_y=12, det_x=64, det_y=64)
print(f"Rectangular scan shape: {data_rect.shape}")

w_rect_scan = Show4DSTEM(data_rect)
w_rect_scan.roi_circle(radius=8.0)
print(f"Scan shape: {w_rect_scan.scan_shape}")
print(f"Detector shape: {w_rect_scan.detector_shape}")
w_rect_scan

## 14. VI ROI (Real-Space Region for Summed DP)

The VI ROI selects a region in the virtual image (real-space) and sums all
diffraction patterns within that region. This produces an averaged diffraction
pattern with better signal-to-noise, useful for identifying crystallographic
features from a specific area of the sample.

In [ ]:
w_vi = Show4DSTEM(data)
w_vi.roi_circle(radius=8.0)

# Enable circular VI ROI in real-space
w_vi.vi_roi_mode = "circle"
w_vi.vi_roi_center_row = 8.0
w_vi.vi_roi_center_col = 8.0
w_vi.vi_roi_radius = 4.0
print(f"VI ROI: circle at ({w_vi.vi_roi_center_row}, {w_vi.vi_roi_center_col}), r={w_vi.vi_roi_radius}")
print(f"Summed {w_vi.summed_dp_count} positions")
w_vi

## 15. Mask DC Component

The central pixel of the diffraction pattern (DC component) often saturates
the detector. `mask_dc` excludes the center 3x3 region from DP statistics
calculations, giving more meaningful contrast metrics.

In [ ]:
# DC masking enabled (default)
w_dc_on = Show4DSTEM(data)
w_dc_on.mask_dc = True
print(f"mask_dc=True  -> DP stats: {w_dc_on.dp_stats}")

# DC masking disabled
w_dc_off = Show4DSTEM(data)
w_dc_off.mask_dc = False
w_dc_off.position = w_dc_on.position  # same position for comparison
print(f"mask_dc=False -> DP stats: {w_dc_off.dp_stats}")
print("\nWith mask_dc=True, the center bright spot is excluded from stats.")
w_dc_on